Preparing on-street parking data - converting polygons into matchable points

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import os
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = 'db'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [ ]:
parking = snman.io.import_geofile_to_gdf(
    "C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/Parkflächen Erhebung Stadt+OSM.gpkg"
).query("typ in ['Parkstreifen', 'Zickzack']")
parking

In [ ]:
import numpy as np
import shapely as shp

def random_points_in_polygon(polygon, number):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = shp.Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return shp.MultiPoint(points)

parking.geometry = parking.apply(
    lambda row: random_points_in_polygon(row.geometry, row['kapazitaet']),
    axis=1
)

In [ ]:
parking = parking.explode().reset_index(drop=True)
parking

In [ ]:
snman.io.export_gdf(
    parking,
    "C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/Parkflächen Erhebung Stadt+OSM_punkte.gpkg"
)